In [1]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()
# Generate a URL for the image thumbnail



Successfully saved authorization token.


In [85]:
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import geemap
import requests
from PIL import Image
import numpy as np

In [87]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define a region of interest (ROI) and scale for the image
roi = ee.Geometry.Rectangle([-122.44, 37.74, -122.34, 37.84])
scale = 30  # Resolution in meters

# Get a thumbnail URL of the image
thumbnail_url = image.getThumbURL({
    'region': roi,
    'scale': scale,
})

# Download the thumbnail image
response = requests.get(thumbnail_url)
img = Image.open(io.BytesIO(response.content))

# Convert the image to a NumPy array
img_array = np.array(img)
G = nx.Graph()

# Iterate through the pixels and add nodes to the graph
for i in range(img_array.shape[0]):
    for j in range(img_array.shape[1]):
        node_value = img_array[i, j]
        G.add_node((i, j), value=node_value)

# Define a function to compute the similarity between pixel values (you can customize this)
def compute_similarity(node1, node2):
    return np.linalg.norm(node1['value'] - node2['value'])

# Add edges between nodes based on similarity (e.g., Euclidean distance)
for node1 in G.nodes(data=True):
    for node2 in G.nodes(data=True):
        if node1 != node2:
            similarity = compute_similarity(node1[1], node2[1])
            if similarity < threshold:  # Define a threshold for edge creation
                G.add_edge(node1[0], node2[0], weight=similarity)
pos = {node: node for node in G.nodes()}
nx.draw(G, pos=pos, with_labels=False, node_size=5)
plt.show()

EEException: Must specify either one or three bands for PNG or JPEG encoding.

In [64]:
lst = ee.ImageCollection('MODIS/006/MOD11A1')
# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

# Define the urban location of interest as a point near Lyon, France.
u_lon = -115.4734
u_lat = 33.6826155
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
'''
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)
'''
roi = u_poi.buffer(1e6)
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

# Create a URL to the styled image for a region around France.
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature in France.
Image(url=url)


https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/9a618d8ed6fd520959629532e143a470-f0b661a9b8e3eccd66f60ed3a88cfb6a:getPixels


In [74]:
biomass = ee.Image('WCMC/biomass_carbon_density/v1_0/2010')
url = biomass.getThumbUrl({
    'min': 1, 'max': 180, 'dimensions': 512,
    'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']})
print(url)
Image(url=url)

https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/62c030bc38dc1871ca9ed22acf9e7375-4980ea8e045f76b3fa86d61de6a42c8b:getPixels


In [97]:
def biomass_val(lon, lat):
    point = ee.Geometry.Point(lon, lat)
    value = biomass.reduceRegion(ee.Reducer.first(), point, 1)
    return value.getInfo()
print(biomass_val(-100,34))

{'carbon_tonnes_per_ha': 23.51944923400879}


In [67]:
ndvi = ee.ImageCollection('MODIS/061/MOD13A2')
# Initial date of interest (inclusive).
i_date = '2018-01-01'

# Final date of interest (exclusive).
f_date = '2018-05-01'

# Selection of appropriate bands and dates for NDVI.
ndvi = ndvi.select('NDVI').filterDate(i_date, f_date)

# Define the urban location of interest as a point near Lyon, France.
u_lon = -115.4734
u_lat = 33.6826
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
'''
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)
'''

roi = u_poi.buffer(1e6)

# Reduce the LST collection by mean.
ndvi_img = ndvi.mean()

# Create a URL to the styled image for a region around California.
url = ndvi_img.getThumbUrl({
    'min': 0, 'max': 9000, 'dimensions': 512, 'region': roi,
    'palette': ['ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301']})
print(url)

# Display the thumbnail land surface temperature in California.
Image(url=url)


https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/6e0ddcd54dde2e3f1609461441a7c471-8478202a33a9e8ac3b9ffefd38e09315:getPixels


In [69]:
# Load an image from Earth Engine (e.g., a land cover image)
image = ee.Image("USGS/GFSAD1000_V1")

# Create a geometry for the region of interest (ROI)
roi = ee.Geometry.Polygon(
    [[[-120, 35], [-120, 36], [-119, 36], [-119, 35]]]
)

# Clip the image to the ROI
clipped_image = image.clip(roi)

# Define the resolution (e.g., 100 meters per pixel)
resolution = 100

# Get the projection information
projection = clipped_image.projection()

# Create a grid of points with 1-hectare spacing
grid = ee.FeatureCollection.randomPoints(roi, 1, 1, 42)

# Convert the grid points to a list
grid_list = grid.toList(grid.size())

# Initialize a graph
graph = nx.Graph()

# Iterate through the grid points
for i in range(grid.size().getInfo()):
    point = ee.Feature(grid_list.get(i))
    x, y = point.geometry().coordinates().getInfo()
    
    # Calculate adjacent nodes (assuming 8-connectivity)
    adjacent_nodes = []
    for dx in [-resolution, 0, resolution]:
        for dy in [-resolution, 0, resolution]:
            if dx == 0 and dy == 0:
                continue
            neighbor_x = x + dx
            neighbor_y = y + dy
            adjacent_nodes.append((neighbor_x, neighbor_y))
    
    # Add the current node and its adjacent nodes to the graph
    graph.add_node((x, y))
    for neighbor in adjacent_nodes:
        if neighbor in graph.nodes():
            graph.add_edge((x, y), neighbor)

# Now, the 'graph' variable contains a NetworkX graph with nodes representing hectares
# and edges connecting adjacent nodes.

# You can perform various graph analysis tasks on 'graph' using NetworkX functions.
# For example, you can calculate centrality, shortest paths, or visualize the graph.

# Example: Calculate the number of nodes and edges in the graph
num_nodes = len(graph.nodes())
num_edges = len(graph.edges())
print(f"Number of nodes: {graph.nodes()}")
print(f"Number of edges: {graph.edges()}")


Number of nodes: [(-119.91574732920664, 35.98122700796177)]
Number of edges: []
